In [42]:
# 라이브러리
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import sys
import os
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

In [18]:
# 경로 및 모듈 로딩
sys.path.append(os.path.abspath('../scripts'))
from model import CNNLSTMModel  # 또는 cnn_lstm_model.py 사용 시

In [19]:
# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 Using device: {device}")

🚀 Using device: cuda


In [ ]:
# 데이터 로딩
X = np.load('../data/processed/X_input.npy')
y = np.load('../data/processed/y_target.npy')

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

In [24]:
# 데이터 분할
train_ratio = 0.8
split_idx = int(len(X_tensor) * train_ratio)
X_train, X_val = X_tensor[:split_idx], X_tensor[split_idx:]
y_train, y_val = y_tensor[:split_idx], y_tensor[split_idx:]

In [30]:
# TensorDataset & DataLoader
batch_size = 64  # or 32 for tighter memory
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [31]:
# 모델 정의
input_dim = X.shape[2]
model = CNNLSTMModel(input_dim=input_dim).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [32]:
# 학습 루프
epochs = 20
for epoch in range(1, epochs + 1):
    model.train()
    train_loss = 0.0

    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        output = model(xb)
        loss = criterion(output, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * xb.size(0)

    train_loss /= len(train_loader.dataset)

    # 🔹 검증
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            output = model(xb)
            loss = criterion(output, yb)
            val_loss += loss.item() * xb.size(0)

    val_loss /= len(val_loader.dataset)

    print(f"[{epoch:02d}/{epochs}] 🟢 Train Loss: {train_loss:.6f} | 🔵 Val Loss: {val_loss:.6f}")

[01/20] 🟢 Train Loss: 0.035623 | 🔵 Val Loss: 0.020719
[02/20] 🟢 Train Loss: 0.001552 | 🔵 Val Loss: 0.010993
[03/20] 🟢 Train Loss: 0.000595 | 🔵 Val Loss: 0.002629
[04/20] 🟢 Train Loss: 0.000130 | 🔵 Val Loss: 0.002572
[05/20] 🟢 Train Loss: 0.000110 | 🔵 Val Loss: 0.002850
[06/20] 🟢 Train Loss: 0.000109 | 🔵 Val Loss: 0.002331
[07/20] 🟢 Train Loss: 0.000100 | 🔵 Val Loss: 0.002035
[08/20] 🟢 Train Loss: 0.000097 | 🔵 Val Loss: 0.002296
[09/20] 🟢 Train Loss: 0.000098 | 🔵 Val Loss: 0.002495
[10/20] 🟢 Train Loss: 0.000093 | 🔵 Val Loss: 0.001950
[11/20] 🟢 Train Loss: 0.000088 | 🔵 Val Loss: 0.002264
[12/20] 🟢 Train Loss: 0.000091 | 🔵 Val Loss: 0.002115
[13/20] 🟢 Train Loss: 0.000089 | 🔵 Val Loss: 0.001560
[14/20] 🟢 Train Loss: 0.000101 | 🔵 Val Loss: 0.001852
[15/20] 🟢 Train Loss: 0.000089 | 🔵 Val Loss: 0.001690
[16/20] 🟢 Train Loss: 0.000095 | 🔵 Val Loss: 0.001601
[17/20] 🟢 Train Loss: 0.000079 | 🔵 Val Loss: 0.001833
[18/20] 🟢 Train Loss: 0.000095 | 🔵 Val Loss: 0.001619
[19/20] 🟢 Train Loss: 0.0000

In [ ]:
# 모델 저장
os.makedirs('../models', exist_ok=True)
torch.save(model.state_dict(), '../models/cnn_lstm_model.pt')
print("✅ 모델 저장 완료: ../models/cnn_lstm_model.pt")

✅ 모델 저장 완료: ../models/cnn_lstm_model.pt


In [35]:
# 성능 지표 평가
model.eval()
y_val_pred = []
y_val_true = []

with torch.no_grad():
    for xb, yb in val_loader:
        xb = xb.to(device)
        pred = model(xb).cpu().numpy()
        y_val_pred.append(pred)
        y_val_true.append(yb.cpu().numpy())

y_val_pred = np.concatenate(y_val_pred, axis=0)
y_val_true = np.concatenate(y_val_true, axis=0)

In [49]:
# 월별 CPI 평균값 기준 평가
y_true_avg = y_val_true.mean(axis=1)
y_pred_avg = y_val_pred.mean(axis=1)

smape = np.mean(np.abs((y_true_avg - y_pred_avg) / y_true_avg)) * 100
rmse = root_mean_squared_error(y_true_avg, y_pred_avg)
mae = mean_absolute_error(y_true_avg, y_pred_avg)
r2 = r2_score(y_true_avg, y_pred_avg)
nrmse = rmse / np.std(y_true_avg)

print(f"📊 성능 지표:")
print(f"SMAPE: {smape:.4f}\nRMSE: {rmse:.4f}\nMAE: {mae:.4f}\nR2: {r2:.4f}\nNRMSE: {nrmse:.4f}")

📊 성능 지표:
SMAPE: 3.9032
RMSE: 0.0445
MAE: 0.0344
R2: 0.5348
NRMSE: 0.6821


In [45]:
# 성능 지표 저장
os.makedirs('../results', exist_ok=True)
with open('../results/metrics.txt', 'w') as f:
    f.write(f"RMSE   : {rmse:.6f}\n")
    f.write(f"MAE    : {mae:.6f}\n")
    f.write(f"R²     : {r2:.4f}\n")
    f.write(f"NRMSE  : {nrmse:.4f}\n")
print("📁 성능 지표 저장 완료: ../results/metrics.txt")

📁 성능 지표 저장 완료: ../results/metrics.txt
